In [1]:
import pandas as pd
import numpy as np
import glob
import re

from pprint import pprint

In [2]:
glob.glob('./Second pass model outputs/*.txt')

['./Second pass model outputs\\ByT5 Console Output.txt',
 './Second pass model outputs\\byt5 inference clean data.txt',
 './Second pass model outputs\\byt5 inference data.txt',
 './Second pass model outputs\\byt5 multi iteration data.txt',
 './Second pass model outputs\\T5 Console Output.txt',
 './Second pass model outputs\\t5 full dateaset word segment.txt',
 './Second pass model outputs\\t5 inference data.txt',
 './Second pass model outputs\\t5 multi iteration 70 percentage data trained on.txt',
 './Second pass model outputs\\t5 multi iteration data segmented answer model.txt',
 './Second pass model outputs\\t5 multi iteration data segmented answer model_2.txt',
 './Second pass model outputs\\t5 multi iteration data word segmented.txt',
 './Second pass model outputs\\t5 multi iteration data.txt',
 './Second pass model outputs\\t5 multi iteration wod segmented 2 epochs.txt',
 './Second pass model outputs\\t5 training output.txt',
 './Second pass model outputs\\t5_small on all dataset 

In [3]:
txt_path = "./Second pass model outputs\\t5_small_new_embedding.txt"

lines = open(txt_path, 'r').readlines()

In [4]:
def extract_float(input_string):
    pattern = r"\d+\.\d+"
    matches = re.findall(pattern, input_string)
    float_numbers = [float(match) for match in matches]
    return float_numbers

In [5]:
date_pattern = r"\b\d{2}/\d{2}/\d{4}\b"
data_dict = {}
new_date_math = False
current_date = ''
error_dates = []

for line in lines:
    if 'error' in line:
        match = re.search(date_pattern, line)
        error_dates.append(match.group())

for line in lines:
    match = re.match(date_pattern, line) # to match the date pattern

    if match:
        current_date = line.strip()
        data_dict[current_date] = {}
        data_dict[current_date]['Letter II'] = []
        data_dict[current_date]['Word II'] = []


    if 'Before' in line:
        [lett_accu, word_accu] = extract_float(line)
        data_dict[current_date]['Before Letter Accuracy'] = lett_accu
        data_dict[current_date]['Before Word Accuracy'] = word_accu

    if 'iteration:' in line:
        lett_accu, word_accu = extract_float(line)
        data_dict[current_date]['Letter II'].append(lett_accu)
        data_dict[current_date]['Word II'].append(word_accu)


In [6]:
output_data = []
for date, inf_data in data_dict.items():
    
    if len(inf_data.keys()) < 4:
        continue
    
    # only first pass model output exists
    if len(inf_data['Letter II']) == 0:
        f_pass_l_accu = inf_data['Before Letter Accuracy']
        f_pass_w_accu = inf_data['Before Word Accuracy']
        
        s_pass_l_accu = inf_data['Before Letter Accuracy']
        s_pass_w_accu = inf_data['Before Word Accuracy']
        output_data.append((date, f_pass_l_accu, f_pass_w_accu, s_pass_l_accu, s_pass_w_accu))
    else:
        f_pass_l_accu = round(inf_data['Before Letter Accuracy'], 2)
        f_pass_w_accu = round(inf_data['Before Word Accuracy'], 2)
        
        s_pass_l_list = inf_data['Letter II']
        s_pass_w_list = inf_data['Word II']
        max_accu_index = s_pass_l_list.index(max(s_pass_l_list))
        
        s_pass_l_accu = round(s_pass_l_list[max_accu_index], 2)
        s_pass_w_accu = round(s_pass_w_list[max_accu_index], 2)
        
        if s_pass_l_accu < f_pass_l_accu:
            s_pass_l_accu = f_pass_l_accu
            s_pass_w_accu = f_pass_w_accu
            
        output_data.append((date, f_pass_l_accu, f_pass_w_accu, s_pass_l_accu, s_pass_w_accu))

df = pd.DataFrame(output_data, columns = ['Date', 'First Pass Letter Accuracy', 'First Pass Word Accuracy', 'Second Pass Letter Accuracy', 'Second Pass Word Accuracy'])
df.to_csv('./Second pass model outputs/t5_with_new_embedding_2023_test_set.csv')